Tento kód byl převzat z https://github.com/simoninithomas/CatDCGAN a upraven pro použití na generování šifer

In [1]:
import os
import json
import numpy as np

import tensorflow.compat.v1 as tf

tf.disable_v2_behavior() 
from helper_ds import *
from glob import glob
import pickle as pkl
import scipy.misc

import time

import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

do_preprocess = False
from_checkpoint = True

2022-12-06 17:29:52.071319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 17:29:52.204745: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-06 17:29:52.506368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jure/miniconda3/envs/barax/lib/
2022-12-06 17:29:52.506416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plug

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def resize_image(img_name, size, source_dir, resized_path):
    if img_name.startswith('.'):
        return
    img_path= source_dir +img_name 

    if not os.path.exists(resized_path):
        os.makedirs(resized_path)

    image=Image.open(img_path)
    resized_image=image.resize((size, size))
    resized_image.save(resized_path+"/"+img_name, "PNG")

In [38]:
IMG_SIZE=128


TRAIN_TEST_ORIGINALS_DIR = 'data/output/train/'
data_dir_h = TRAIN_TEST_ORIGINALS_DIR+'hard/' # Data
data_dir_e = TRAIN_TEST_ORIGINALS_DIR+'easy/' # Data

data_resized_dir = "data/hard_resized_data_128"# Resized data



if do_preprocess == True:
    if not os.path.exists(data_resized_dir):
        os.makedirs(data_resized_dir) 

    for image in os.listdir(data_dir_h):
        resize_image(image, IMG_SIZE, data_dir_h, data_resized_dir)    

"\n    for image in os.listdir(data_dir_e):\n        resize_image(image, IMG_SIZE, data_dir_e, data_resized_dir)\n\n\n    TRAIN_TEST_ORIGINALS_DIR = 'data/quick_draw_dataset/'\n    for image in os.listdir(TRAIN_TEST_ORIGINALS_DIR):\n        #print(image)\n        resize_image(image, IMG_SIZE, TRAIN_TEST_ORIGINALS_DIR, data_resized_dir)\n\n\n\n"

In [39]:
# This part was taken from Udacity Face generator project
def get_image(image_path, width, height, mode):
    """
    Read image from image_path
    :param image_path: Path of image
    :param width: Width of image
    :param height: Height of image
    :param mode: Mode of image
    :return: Image data
    """
    image = Image.open(image_path)

    return np.array(image.convert(mode))

def get_batch(image_files, width, height, mode):
    data_batch = np.array(
        [get_image(sample_file, width, height, mode) for sample_file in image_files]).astype(np.float32)

    # Make sure the images are in 4 dimensions
    if len(data_batch.shape) < 4:
        data_batch = data_batch.reshape(data_batch.shape + (1,))

    return data_batch



In [40]:
def model_inputs(real_dim, z_dim):
    """
    Create the model inputs
    :param real_dim: tuple containing width, height and channels
    :param z_dim: The dimension of Z
    :return: Tuple of (tensor of real input images, tensor of z data, learning rate G, learning rate D)
    """
    inputs_real = tf.placeholder(tf.float32, (None, *real_dim), name='inputs_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name="input_z")
    learning_rate_G = tf.placeholder(tf.float32, name="learning_rate_G")
    learning_rate_D = tf.placeholder(tf.float32, name="learning_rate_D")
    
    return inputs_real, inputs_z, learning_rate_G, learning_rate_D     

In [41]:
def generator(z, output_channel_dim, is_train=True):
    ''' Build the generator network.
    
        Arguments
        ---------
        z : Input tensor for the generator
        output_channel_dim : Shape of the generator output
        n_units : Number of units in hidden layer
        reuse : Reuse the variables with tf.variable_scope
        alpha : leak parameter for leaky ReLU
        
        Returns
        -------
        out: 
    '''
    with tf.variable_scope("generator", reuse= not is_train):
        
        # First FC layer --> 8x8x1024
        fc1 = tf.layers.dense(z, 8*8*1024)
        
        # Reshape it
        fc1 = tf.reshape(fc1, (-1, 8, 8, 1024))
        
        # Leaky ReLU
        fc1 = tf.nn.leaky_relu(fc1, alpha=alpha)

        
        # Transposed conv 1 --> BatchNorm --> LeakyReLU
        # 8x8x1024 --> 16x16x512
        trans_conv1 = tf.layers.conv2d_transpose(inputs = fc1,
                                  filters = 512,
                                  kernel_size = [5,5],
                                  strides = [2,2],
                                  padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name="trans_conv1")
        
        batch_trans_conv1 = tf.layers.batch_normalization(inputs = trans_conv1, training=is_train, epsilon=1e-5, name="batch_trans_conv1")
       
        trans_conv1_out = tf.nn.leaky_relu(batch_trans_conv1, alpha=alpha, name="trans_conv1_out")
        
        
        # Transposed conv 2 --> BatchNorm --> LeakyReLU
        # 16x16x512 --> 32x32x256
        trans_conv2 = tf.layers.conv2d_transpose(inputs = trans_conv1_out,
                                  filters = 256,
                                  kernel_size = [5,5],
                                  strides = [2,2],
                                  padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name="trans_conv2")
        
        batch_trans_conv2 = tf.layers.batch_normalization(inputs = trans_conv2, training=is_train, epsilon=1e-5, name="batch_trans_conv2")
       
        trans_conv2_out = tf.nn.leaky_relu(batch_trans_conv2, alpha=alpha, name="trans_conv2_out")
        
        
        # Transposed conv 3 --> BatchNorm --> LeakyReLU
        # 32x32x256 --> 64x64x128
        trans_conv3 = tf.layers.conv2d_transpose(inputs = trans_conv2_out,
                                  filters = 128,
                                  kernel_size = [5,5],
                                  strides = [2,2],
                                  padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name="trans_conv3")
        
        batch_trans_conv3 = tf.layers.batch_normalization(inputs = trans_conv3, training=is_train, epsilon=1e-5, name="batch_trans_conv3")
       
        trans_conv3_out = tf.nn.leaky_relu(batch_trans_conv3, alpha=alpha, name="trans_conv3_out")

        
        # Transposed conv 4 --> BatchNorm --> LeakyReLU
        # 64x64x128 --> 128x128x64
        trans_conv4 = tf.layers.conv2d_transpose(inputs = trans_conv3_out,
                                  filters = 64,
                                  kernel_size = [5,5],
                                  strides = [2,2],
                                  padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name="trans_conv4")
        
        batch_trans_conv4 = tf.layers.batch_normalization(inputs = trans_conv4, training=is_train, epsilon=1e-5, name="batch_trans_conv4")
       
        trans_conv4_out = tf.nn.leaky_relu(batch_trans_conv4, alpha=alpha, name="trans_conv4_out")

        
        # Transposed conv 5 --> tanh
        # 128x128x64 --> 128x128x3
        logits = tf.layers.conv2d_transpose(inputs = trans_conv4_out,
                                  filters = 3,
                                  kernel_size = [5,5],
                                  strides = [1,1],
                                  padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name="logits")
         
        out = tf.tanh(logits, name="out")
        
        return out


In [42]:
def discriminator(x, is_reuse=False, alpha = 0.2):
    ''' Build the discriminator network.
    
        Arguments
        ---------
        x : Input tensor for the discriminator
        n_units: Number of units in hidden layer
        reuse : Reuse the variables with tf.variable_scope
        alpha : leak parameter for leaky ReLU
        
        Returns
        -------
        out, logits: 
    '''
    with tf.variable_scope("discriminator", reuse = is_reuse): 
        
        # Input layer 128*128*3 --> 64x64x64
        # Conv --> BatchNorm --> LeakyReLU   
        conv1 = tf.layers.conv2d(inputs = x,
                                filters = 64,
                                kernel_size = [5,5],
                                strides = [2,2],
                                padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name='conv1')
        
        batch_norm1 = tf.layers.batch_normalization(conv1,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm1')

        conv1_out = tf.nn.leaky_relu(batch_norm1, alpha=alpha, name="conv1_out")
        
        
        # 64x64x64--> 32x32x128
        # Conv --> BatchNorm --> LeakyReLU   
        conv2 = tf.layers.conv2d(inputs = conv1_out,
                                filters = 128,
                                kernel_size = [5, 5],
                                strides = [2, 2],
                                padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name='conv2')
        
        batch_norm2 = tf.layers.batch_normalization(conv2,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm2')
        
        conv2_out = tf.nn.leaky_relu(batch_norm2, alpha=alpha, name="conv2_out")

        
        
        # 32x32x128 --> 16x16x256
        # Conv --> BatchNorm --> LeakyReLU   
        conv3 = tf.layers.conv2d(inputs = conv2_out,
                                filters = 256,
                                kernel_size = [5, 5],
                                strides = [2, 2],
                                padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name='conv3')
        
        batch_norm3 = tf.layers.batch_normalization(conv3,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                name = 'batch_norm3')
        
        conv3_out = tf.nn.leaky_relu(batch_norm3, alpha=alpha, name="conv3_out")

        
        
        # 16x16x256 --> 16x16x512
        # Conv --> BatchNorm --> LeakyReLU   
        conv4 = tf.layers.conv2d(inputs = conv3_out,
                                filters = 512,
                                kernel_size = [5, 5],
                                strides = [1, 1],
                                padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name='conv4')
        
        batch_norm4 = tf.layers.batch_normalization(conv4,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                name = 'batch_norm4')
        
        conv4_out = tf.nn.leaky_relu(batch_norm4, alpha=alpha, name="conv4_out")

        
        
        # 16x16x512 --> 8x8x1024
        # Conv --> BatchNorm --> LeakyReLU   
        conv5 = tf.layers.conv2d(inputs = conv4_out,
                                filters = 1024,
                                kernel_size = [5, 5],
                                strides = [2, 2],
                                padding = "SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                name='conv5')
        
        batch_norm5 = tf.layers.batch_normalization(conv5,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                name = 'batch_norm5')
        
        conv5_out = tf.nn.leaky_relu(batch_norm5, alpha=alpha, name="conv5_out")

         
        # Flatten it
        flatten = tf.reshape(conv5_out, (-1, 8*8*1024))
        
        # Logits
        logits = tf.layers.dense(inputs = flatten,
                                units = 1,
                                activation = None)
        
        
        out = tf.sigmoid(logits)
        
        return out, logits

In [43]:
def model_loss(input_real, input_z, output_channel_dim, alpha):
    """
    Get the loss for the discriminator and generator
    :param input_real: Images from the real dataset
    :param input_z: Z input
    :param out_channel_dim: The number of channels in the output image
    :return: A tuple of (discriminator loss, generator loss)
    """
    # Generator network here
    g_model = generator(input_z, output_channel_dim)   
    # g_model is the generator output
    
    # Discriminator network here
    d_model_real, d_logits_real = discriminator(input_real, alpha=alpha)
    d_model_fake, d_logits_fake = discriminator(g_model,is_reuse=True, alpha=alpha)
    
    # Calculate losses
    d_loss_real = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, 
                                                          labels=tf.ones_like(d_model_real)))
    d_loss_fake = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, 
                                                          labels=tf.zeros_like(d_model_fake)))
    d_loss = d_loss_real + d_loss_fake

    g_loss = tf.reduce_mean(
             tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                     labels=tf.ones_like(d_model_fake)))
    
    return d_loss, g_loss

In [44]:
def model_optimizers(d_loss, g_loss, lr_D, lr_G, beta1):
    """
    Get optimization operations
    :param d_loss: Discriminator loss Tensor
    :param g_loss: Generator loss Tensor
    :param learning_rate: Learning Rate Placeholder
    :param beta1: The exponential decay rate for the 1st moment in the optimizer
    :return: A tuple of (discriminator training operation, generator training operation)
    """    
    # Get the trainable_variables, split into G and D parts
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith("generator")]
    d_vars = [var for var in t_vars if var.name.startswith("discriminator")]
    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    # Generator update
    gen_updates = [op for op in update_ops if op.name.startswith('generator')]
    
    # Optimizers
    with tf.control_dependencies(gen_updates):
        d_train_opt = tf.train.AdamOptimizer(learning_rate=lr_D, beta1=beta1).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(learning_rate=lr_G, beta1=beta1).minimize(g_loss, var_list=g_vars)
        
    return d_train_opt, g_train_opt

In [45]:
def show_generator_output(sess, n_images, input_z, out_channel_dim, image_mode, image_path, save, show):
    """
    Show example output for the generator
    :param sess: TensorFlow session
    :param n_images: Number of Images to display
    :param input_z: Input Z Tensor
    :param out_channel_dim: The number of channels in the output image
    :param image_mode: The mode to use for images ("RGB" or "L")
    :param image_path: Path to save the image
    """
    cmap = None if image_mode == 'RGB' else 'gray'
    z_dim = input_z.get_shape().as_list()[-1]
    example_z = np.random.uniform(-1, 1, size=[n_images, z_dim])

    samples = sess.run(
        generator(input_z, out_channel_dim, False),
        feed_dict={input_z: example_z})

    images_grid = images_square_grid(samples, image_mode)
    
    if save == True:
        # Save image
        images_grid.save(image_path, 'JPEG')
    
    if show == True:
        plt.imshow(images_grid, cmap=cmap)
        plt.show()

In [59]:
def train(epoch_count, batch_size, z_dim, learning_rate_D, learning_rate_G, beta1, dataset, alpha, from_checkpoint):
    """
    Train the GAN
    :param epoch_count: Number of epochs
    :param batch_size: Batch Size
    :param z_dim: Z dimension
    :param learning_rate: Learning Rate
    :param beta1: The exponential decay rate for the 1st moment in the optimizer
    :param get_batches: Function to get batches
    :param data_shape: Shape of the data
    :param data_image_mode: The image mode to use for images ("RGB" or "L")
    """
    
    losses=[]
    # Create our input placeholders
    input_images, input_z, lr_G, lr_D = model_inputs(dataset.shape[1:], z_dim)
        
    # Losses
    d_loss, g_loss = model_loss(input_images, input_z, dataset.shape[3], alpha)
    losses.append([d_loss])
    losses.append([g_loss])
    
    # Optimizers
    d_opt, g_opt = model_optimizers(d_loss, g_loss, lr_D, lr_G, beta1)
    
    i = 0
    j=0
    
    genrated_img_path='./data/generated_images_hard_ciphers/'
    if not os.path.exists(genrated_img_path):
        os.makedirs(genrated_img_path)
    
    version = "firstTrain"
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        # Saver
        saver = tf.train.Saver()
        
        num_epoch = 0
        
        if from_checkpoint == True:
            saver.restore(sess, "data/model_qd_2/model.ckpt")
            
            #show_generator_output(sess, 4, input_z, dataset.shape[3], dataset.image_mode, image_path, True, False)
        for epoch_i in range(0, epoch_count):
            print(epoch_count)
            num_epoch =epoch_i

            j+=1
            print(j)
            if (num_epoch+1) % 5 == 0:

                # Save model every 5 epochs
                #if not os.path.exists("models/" + version):
                #    os.makedirs("models/" + version)
                save_path = saver.save(sess, "data/model_hard_ciphers/model.ckpt")
                print("Model saved")

            for batch_images in dataset.get_batches(batch_size):
                # Random noise
                batch_z = np.random.uniform(-1, 1, size=(batch_size, z_dim))
                i += 1

                # Run optimizers
                _ = sess.run(d_opt, feed_dict={input_images: batch_images, input_z: batch_z, lr_D: learning_rate_D})
                _ = sess.run(g_opt, feed_dict={input_images: batch_images, input_z: batch_z, lr_G: learning_rate_G})

                if i % 10 == 0:
                    train_loss_d = d_loss.eval({input_z: batch_z, input_images: batch_images})
                    train_loss_g = g_loss.eval({input_z: batch_z})
                    losses[0].append(d_loss)
                    losses[1].append(g_loss)

                    # Save it
                    image_name = str(epoch_i+1) + '_'+str(i) + ".jpg"
                    image_path = genrated_img_path + image_name
                    show_generator_output(sess, 4, input_z, dataset.shape[3], dataset.image_mode, image_path, True, False) 

                # Print every 5 epochs (for stability overwize the jupyter notebook will bug)
                if i % 1500 == 0:

                    image_name = str(i) + ".jpg"
                    image_path = genrated_img_path + image_name
                    print("Epoch {}/{}...".format(epoch_i+1, epochs),
                          "Discriminator Loss: {:.4f}...".format(train_loss_d),
                          "Generator Loss: {:.4f}".format(train_loss_g))
                    show_generator_output(sess, 4, input_z, dataset.shape[3], dataset.image_mode, image_path, False, True)

            
    return losses

In [60]:
# Size input image for discriminator
real_size = (128,128,3)

# Size of latent vector to generator
z_dim = 100
learning_rate_D =  .00000005 # Thanks to Alexia Jolicoeur Martineau https://ajolicoeur.wordpress.com/cats/
learning_rate_G = 3e-4 # Thanks to Alexia Jolicoeur Martineau https://ajolicoeur.wordpress.com/cats/
batch_size = 20
epochs = 40
alpha = 0.2
beta1 = 0.5

# Create the network
#model = DGAN(real_size, z_size, learning_rate, alpha, beta1)

In [61]:

# Load the data and train the network here
dataset = Dataset(glob(os.path.join(data_resized_dir, '*.png')))

with tf.Graph().as_default():
    losses, samples = train(epochs, batch_size, z_dim, learning_rate_D, learning_rate_G, beta1, dataset, alpha, from_checkpoint)


/tmp/ipykernel_1208389/74336796.py:19: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  fc1 = tf.layers.dense(z, 8*8*1024)
/tmp/ipykernel_1208389/74336796.py:30: UserWarning: `tf.layers.conv2d_transpose` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2DTranspose` instead.
  trans_conv1 = tf.layers.conv2d_transpose(inputs = fc1,
/tmp/ipykernel_1208389/74336796.py:38: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  batch_trans_conv1 = tf.layers.batch_normalization(inputs = trans_conv1, training=is_train, epsilon=1e-5, name="batch_trans_conv1")
/tmp/ipykernel_1208389/74336796.py:45: UserWarning: `tf.layers.conv

INFO:tensorflow:Restoring parameters from data/model_qd_2/model.ckpt
40
1
40
2
40
3
40
4
40
5
Model saved
40
6
40
7
40
8
40
9
40
10
Model saved
40
11
40
12
40
13
40
14
40
15
Model saved
40
16
40
17
40
18
40
19
40
20
Model saved
40
21
40
22
40
23
40
24
40
25
Model saved
40
26
40
27
40
28
40
29
40
30
Model saved
40
31
40
32
40
33
40
34
40
35
Model saved
40
36
40
37
40
38
40
39
40
40
Model saved


In [ ]:
from quickdraw import QuickDrawData, QuickDrawDataGroup
qd = QuickDrawData()

categories=qd.drawing_names

QD_DIR='data/quick_draw_dataset/'

In [258]:
img_cnt=0
for c in categories:
    cat=QuickDrawDataGroup(c, max_drawings=150)
    for d in range(cat.drawing_count):
        img=cat.get_drawing
        img.image
    img_cnt+=cat.drawing_count

loading aircraft carrier drawings
load complete
downloading airplane from https://storage.googleapis.com/quickdraw_dataset/full/binary/airplane.bin
download complete
loading airplane drawings
load complete
downloading alarm clock from https://storage.googleapis.com/quickdraw_dataset/full/binary/alarm clock.bin
download complete
loading alarm clock drawings
load complete
downloading ambulance from https://storage.googleapis.com/quickdraw_dataset/full/binary/ambulance.bin
download complete
loading ambulance drawings
load complete
downloading angel from https://storage.googleapis.com/quickdraw_dataset/full/binary/angel.bin
download complete
loading angel drawings
load complete
downloading animal migration from https://storage.googleapis.com/quickdraw_dataset/full/binary/animal migration.bin
download complete
loading animal migration drawings
load complete
downloading ant from https://storage.googleapis.com/quickdraw_dataset/full/binary/ant.bin
download complete
loading ant drawings
load c

download complete
loading camel drawings
load complete
downloading camera from https://storage.googleapis.com/quickdraw_dataset/full/binary/camera.bin
download complete
loading camera drawings
load complete
downloading camouflage from https://storage.googleapis.com/quickdraw_dataset/full/binary/camouflage.bin
download complete
loading camouflage drawings
load complete
downloading campfire from https://storage.googleapis.com/quickdraw_dataset/full/binary/campfire.bin
download complete
loading campfire drawings
load complete
downloading candle from https://storage.googleapis.com/quickdraw_dataset/full/binary/candle.bin
download complete
loading candle drawings
load complete
downloading cannon from https://storage.googleapis.com/quickdraw_dataset/full/binary/cannon.bin
download complete
loading cannon drawings
load complete
downloading canoe from https://storage.googleapis.com/quickdraw_dataset/full/binary/canoe.bin
download complete
loading canoe drawings
load complete
downloading car fr

download complete
loading face drawings
load complete
downloading fan from https://storage.googleapis.com/quickdraw_dataset/full/binary/fan.bin
download complete
loading fan drawings
load complete
downloading feather from https://storage.googleapis.com/quickdraw_dataset/full/binary/feather.bin
download complete
loading feather drawings
load complete
downloading fence from https://storage.googleapis.com/quickdraw_dataset/full/binary/fence.bin
download complete
loading fence drawings
load complete
downloading finger from https://storage.googleapis.com/quickdraw_dataset/full/binary/finger.bin
download complete
loading finger drawings
load complete
downloading fire hydrant from https://storage.googleapis.com/quickdraw_dataset/full/binary/fire hydrant.bin
download complete
loading fire hydrant drawings
load complete
downloading fireplace from https://storage.googleapis.com/quickdraw_dataset/full/binary/fireplace.bin
download complete
loading fireplace drawings
load complete
downloading fire

download complete
loading keyboard drawings
load complete
downloading knee from https://storage.googleapis.com/quickdraw_dataset/full/binary/knee.bin
download complete
loading knee drawings
load complete
downloading knife from https://storage.googleapis.com/quickdraw_dataset/full/binary/knife.bin
download complete
loading knife drawings
load complete
downloading ladder from https://storage.googleapis.com/quickdraw_dataset/full/binary/ladder.bin
download complete
loading ladder drawings
load complete
downloading lantern from https://storage.googleapis.com/quickdraw_dataset/full/binary/lantern.bin
download complete
loading lantern drawings
load complete
downloading laptop from https://storage.googleapis.com/quickdraw_dataset/full/binary/laptop.bin
download complete
loading laptop drawings
load complete
downloading leaf from https://storage.googleapis.com/quickdraw_dataset/full/binary/leaf.bin
download complete
loading leaf drawings
load complete
downloading leg from https://storage.googl

downloading pear from https://storage.googleapis.com/quickdraw_dataset/full/binary/pear.bin
download complete
loading pear drawings
load complete
downloading peas from https://storage.googleapis.com/quickdraw_dataset/full/binary/peas.bin
download complete
loading peas drawings
load complete
downloading pencil from https://storage.googleapis.com/quickdraw_dataset/full/binary/pencil.bin
download complete
loading pencil drawings
load complete
downloading penguin from https://storage.googleapis.com/quickdraw_dataset/full/binary/penguin.bin
download complete
loading penguin drawings
load complete
downloading piano from https://storage.googleapis.com/quickdraw_dataset/full/binary/piano.bin
download complete
loading piano drawings
load complete
downloading pickup truck from https://storage.googleapis.com/quickdraw_dataset/full/binary/pickup truck.bin
download complete
loading pickup truck drawings
load complete
downloading picture frame from https://storage.googleapis.com/quickdraw_dataset/fu

download complete
loading smiley face drawings
load complete
downloading snail from https://storage.googleapis.com/quickdraw_dataset/full/binary/snail.bin
download complete
loading snail drawings
load complete
downloading snake from https://storage.googleapis.com/quickdraw_dataset/full/binary/snake.bin
download complete
loading snake drawings
load complete
downloading snorkel from https://storage.googleapis.com/quickdraw_dataset/full/binary/snorkel.bin
download complete
loading snorkel drawings
load complete
downloading snowflake from https://storage.googleapis.com/quickdraw_dataset/full/binary/snowflake.bin
download complete
loading snowflake drawings
load complete
downloading snowman from https://storage.googleapis.com/quickdraw_dataset/full/binary/snowman.bin
download complete
loading snowman drawings
load complete
downloading soccer ball from https://storage.googleapis.com/quickdraw_dataset/full/binary/soccer ball.bin
download complete
loading soccer ball drawings
load complete
dow

download complete
loading tornado drawings
load complete
downloading tractor from https://storage.googleapis.com/quickdraw_dataset/full/binary/tractor.bin
download complete
loading tractor drawings
load complete
downloading traffic light from https://storage.googleapis.com/quickdraw_dataset/full/binary/traffic light.bin
download complete
loading traffic light drawings
load complete
downloading train from https://storage.googleapis.com/quickdraw_dataset/full/binary/train.bin
download complete
loading train drawings
load complete
downloading tree from https://storage.googleapis.com/quickdraw_dataset/full/binary/tree.bin
download complete
loading tree drawings
load complete
downloading triangle from https://storage.googleapis.com/quickdraw_dataset/full/binary/triangle.bin
download complete
loading triangle drawings
load complete
downloading trombone from https://storage.googleapis.com/quickdraw_dataset/full/binary/trombone.bin
download complete
loading trombone drawings
load complete
down

In [45]:
losses=[]
# Create our input placeholders

version = "firstTrain"
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Saver
    saver = tf.train.Saver()

    num_epoch = 0
    saver.restore(sess, "data/model_qd/model.ckpt")

2022-12-05 14:09:40.192136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-05 14:09:40.192520: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-05 14:09:40.192751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-05 14:09:40.193009: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-05 14:09:40.193211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

INFO:tensorflow:Restoring parameters from data/model_qd/model.ckpt
